In [1]:
%run ./BillableCompanies

In [2]:
notfications = spark.read.format('delta').load("/mnt/bronze/Samples/ATT3ppNotificationsDelta")

In [3]:
display(notfications.limit(5))

ATT3PPNotificationId TranId TranIdChecksum Att3ppNotificationDateTime SubscriberNumber AccountName ResponseURL MDN BAN PrevMDN PrevBAN Status RequestText ResponseSent LatestTicketDate Unclaimed_SentTo Unclaimed_UTCEmailSent ModificationDate FAN CreationDate year month day env 1042382 TPP154880640583700_1_Actsoft2$p1tpp1m6 1234215508 2019-01-30T00:00:06.000+0000 N_NBI_STA_1547583908000138002358 REPUBLIC SERVICES INC CONNECT SMARTPHONES EAST https://mim.cingular.com:443/tpp/absresponse 14343298945 287282389200 null 287282389200 2 REPUBLIC SERVICES INC Mr. JASON PROUTY Telecommunications Mgr 7069220355 fly4151@aol.com Third Party AOP KIRSTIN HEINRICH Telecom Support 9547338700 kirstinh@gcomsinc.com Third Party AOP Jennifer Layton Telecommunications Mgr 7069226250 jlayton@vwpcs.com Third Party AOP Mr./Ms. Laurie Rozendal 4807186248 lrozendal@republicservices.com Day to Day Mr./Ms. Laurie Rozendal 4807186248 lrozendal@republicservices.com Decision Maker Mr./Ms. Laurie Rozendal 4807186248 lrozendal@republicservices.com Initial Premier User Will Barnes Vision Wireless 7069224729 wbarnes@vwpcs.com Third Party Help Desk Peter Frayer Analyst 4807186696 pfrayer@republicservices.com Decision Maker Chris Walker 7069220354 cwalker@vwpcs.com Third Party AOP Stephanie Herring 7069220361 sherring@vwpcs.com Third Party AOP Generic Mailbox 4807186248 republic@vwpcs.com Third Party AOP Janice Hebert 4804931591 jhebert@republicservices.com Authorized Order Placer Samantha Trobough 8557425930 strobough@vwpcs.com Third Party AOP Kevin Jost UNKNOWN 4806272885 kjost@republicservice.com Authorized Order Placer Kathryn Jackson Technology Coordinator 4809992011 kjackson23fc06@republicservices.com Authorized Order Placer Laura Humble 4804931590 lhumble@republicservices.com Authorized Order Placer Jessica Williams 9319807211 jwilliams@vwpcs.com Third Party AOP Josh Darwin 7064149614 jdarwin@vwpcs.com Third Party AOP Jodie Uhl 7067268510 juhl@vwpcs.com Third Party AOP 2019-01-30T00:00:08.860+0000 null null null 2019-01-30T00:00:08.870+0000 04982323 2019-01-30T00:00:08.860+0000 2019 1 30 ATT 1042383 TPP154880641696400_1_Actsoft2$p1tpp1m8 -1701797021 2019-01-30T00:00:17.000+0000 N_NBI_707_1547584110000138002498 REPUBLIC SERVICES INC CONNECT SMARTPHONES EAST https://mim.cingular.com:443/tpp/absresponse 18288293015 287282389200 null 287282389200 2 REPUBLIC SERVICES INC Mr. JASON PROUTY Telecommunications Mgr 7069220355 fly4151@aol.com Third Party AOP KIRSTIN HEINRICH Telecom Support 9547338700 kirstinh@gcomsinc.com Third Party AOP Jennifer Layton Telecommunications Mgr 7069226250 jlayton@vwpcs.com Third Party AOP Mr./Ms. Laurie Rozendal 4807186248 lrozendal@republicservices.com Day to Day Mr./Ms. Laurie Rozendal 4807186248 lrozendal@republicservices.com Decision Maker Mr./Ms. Laurie Rozendal 4807186248 lrozendal@republicservices.com Initial Premier User Will Barnes Vision Wireless 7069224729 wbarnes@vwpcs.com Third Party Help Desk Peter Frayer Analyst 4807186696 pfrayer@republicservices.com Decision Maker Chris Walker 7069220354 cwalker@vwpcs.com Third Party AOP Stephanie Herring 7069220361 sherring@vwpcs.com Third Party AOP Generic Mailbox 4807186248 republic@vwpcs.com Third Party AOP Janice Hebert 4804931591 jhebert@republicservices.com Authorized Order Placer Samantha Trobough 8557425930 strobough@vwpcs.com Third Party AOP Kevin Jost UNKNOWN 4806272885 kjost@republicservice.com Authorized Order Placer Kathryn Jackson Technology Coordinator 4809992011 kjackson23fc06@republicservices.com Authorized Order Placer Laura Humble 4804931590 lhumble@republicservices.com Authorized Order Placer Jessica Williams 9319807211 jwilliams@vwpcs.com Third Party AOP Josh Darwin 7064149614 jdarwin@vwpcs.com Third Party AOP Jodie Uhl 7067268510 juhl@vwpcs.com Third Party AOP 2019-01-30T00:00:19.080+0000 null null null 2019-01-30T00:00:19.090+0000 04982323 2019-01-30T00:00:19.080+0000 2019 1 30 ATT 1042384 TPP154880646211900_1_Actsoft2$p1tpp1m8 1871752363 2019-01-30T00:01:02.000+

In [4]:
from pyspark.sql.column import Column, _to_java_column
from pyspark.sql.types import _parse_datatype_json_string

def ext_from_xml(xml_column, schema, options={}):
    java_column = _to_java_column(xml_column)
    java_schema = spark._jsparkSession.parseDataType(schema.json())
    scala_map = spark._jvm.org.apache.spark.api.python.PythonUtils.toScalaMap(options)
    jc = spark._jvm.com.databricks.spark.xml.functions.from_xml(
        java_column, java_schema, scala_map)
    return Column(jc)

def ext_schema_of_xml_df(df, options={}):
    assert len(df.columns) == 1

    scala_options = spark._jvm.PythonUtils.toScalaMap(options)
    java_xml_module = getattr(getattr(
        spark._jvm.com.databricks.spark.xml, "package$"), "MODULE$")
    java_schema = java_xml_module.schema_of_xml_df(df._jdf, scala_options)
    return _parse_datatype_json_string(java_schema.json())

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import xml.etree.ElementTree as ET
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType


schema = StructType(
    [
      StructField('Header ', StringType(), True), 
      StructField('Order', StringType(), True)
    ]
)

In [6]:
%scala

import com.databricks.spark.xml.functions.from_xml
import com.databricks.spark.xml.schema_of_xml

In [7]:
notfications.count()

Out[7]: 569428

In [8]:
payloadSchema = ext_schema_of_xml_df(notfications.limit(10000).select("RequestText"))


--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1369061821188328> in <module> 
 ----> 1 payloadSchema = ext_schema_of_xml_df ( notfications . limit ( 10000 ) . select ( "RequestText" ) ) 

 <command-1369061821188309> in ext_schema_of_xml_df (df, options) 
 16 java_xml_module = getattr(getattr(
 17 spark._jvm.com.databricks.spark.xml, "package$"), "MODULE$")
 ---> 18 java_schema = java_xml_module . schema_of_xml_df ( df . _jdf , scala_options ) 
 19 return _parse_datatype_json_string ( java_schema . json ( ) ) 

 TypeError : 'JavaPackage' object is not callable

In [9]:
payloadSchema


In [10]:
 parsed = notfications.withColumn("parsed", ext_from_xml("RequestText", payloadSchema))

In [11]:
display(parsed.limit(5))

In [12]:
import pyspark.sql.functions as explode

In [13]:
parsedWColumns = parsed.withColumn('Make', parsed.parsed.Order.Equipment.Device._Make).withColumn('Model', parsed.parsed.Order.Equipment.Device._Model).withColumn('Products', parsed.parsed.Order.Products.Product)
parsedWColumnsProducts = parsedWColumns.withColumn('FirstProduct', parsedWColumns.Products[0]).withColumn('SecondProduct', parsedWColumns.Products[1]).withColumn('ThirdProduct', parsedWColumns.Products[2]).withColumn('FourthProduct', parsedWColumns.Products[3])

In [14]:
from pyspark.sql.functions import col, to_date

wAddProducts = parsedWColumnsProducts.filter((col('FirstProduct')._Action == "Add") | (col('SecondProduct')._Action != "Add") | (col('ThirdProduct')._Action != "Add") | (col('FourthProduct')._Action != "Add")).withColumn('NotificationDate', to_date('Att3ppNotificationDateTime'))

In [15]:
byProductActionType = wAddProducts.select('NotificationDate', 'Make', col('FirstProduct')._Action.alias('First'), col('SecondProduct')._Action.alias('Second'), col('ThirdProduct')._Action.alias('Third'), col('FourthProduct')._Action.alias('Fourth'))

In [16]:
display(wAddProducts.groupBy('NotificationDate', 'Make', "Model").count())

In [17]:
combLicenses = Licenses.join(PaymentInfo, Licenses.PaymentScheduleId == PaymentInfo.PaymentScheduleId)

In [18]:
licWProducts = combLicenses.join(wAddProducts, combLicenses.PTN == wAddProducts.MDN)

In [19]:
licWProducts.count()

In [20]:
licWProducts = licWProducts.withColumn('CreatedDate', to_date(combLicenses.CreatedOn))

In [21]:
display(licWProducts.select(combLicenses.CreatedOn, 'Make'))

In [22]:
wAddProducts